# __REGRESI BOOTSTRAP & JACKNIFE__

----------------------------------------------------------------------------------------

### ___Made By :___

#### _5003221001 - M. Farhan Lucky P<br>5003221061 - M. Iqbal Nurrifki<br>5003221111 - T. Ahyaul Kamal<br>5003221120 - Rasendra Akbar S<br>5003221131 - Ketut Bagus Ananta<br>5003221140 - M. Noor Satriya<br>5003221151 - Yansen Demos Putra N_

#### Link Data yang digunakan : __https://drive.google.com/drive/folders/159dRUHZC3h8aGNmkPsbR4g1CdglU7wHJ?usp=sharing__

------------------------------------------------------------------------------------------------

In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate

## _BOOTSTRAP - XY_

In [31]:
def xy_bootstrap(x, y, n_bootstraps):
    
    def hitung_beta(data_x, data_y):
        matriks_x = np.column_stack((np.ones(data_x.shape[0]), data_x))
        b = np.linalg.pinv(matriks_x.T @ matriks_x) @ (matriks_x.T @ data_y)
        return b
    
    banyak_baris = y.shape[0]
    banyak_kolom = x.shape[1]
    
    true_b = hitung_beta(x, y)

    data_xy = np.column_stack((x,y))
    
    coefs_boot = []
    
    for i in range(n_bootstraps):
        data_boot = data_xy[np.random.choice(data_xy.shape[0], data_xy.shape[0], replace=True)]
        x_bootstrap = data_boot[:, :-1]
        y_bootstrap = data_boot[:, -1]
        
        model = hitung_beta(x_bootstrap, y_bootstrap)
        
        coefs_boot.append(model)
        
    coef_mean = np.mean(coefs_boot, axis=0)
    
    # OUTPUT
    nama_baris = [f'β{i}' for i in range(len(true_b))]
    nama_kolom = ["True Coefficients", "Boostrap Coefficients", "Bias"]
    bias = coef_mean - true_b
    isi_tabel = np.column_stack((true_b, coef_mean, bias))
    output_df = pd.DataFrame(isi_tabel, index=nama_baris, columns=nama_kolom)
    tabel_output = tabulate(output_df, headers='keys', tablefmt='fancy_grid', numalign="center")
    
    print("==================================================================")
    print("\t\t\tXY BOOTSTRAP")
    print("==================================================================")
    print(tabel_output)

In [32]:
data = pd.read_excel("Data_Tugas_Anreg_M13.xlsx")
datax1 = data['X1']
datax2 = data['X2']
datax3 = data['X3']
datax4 = data['X4']
datay = data['Y']
data_x = np.column_stack((datax1, datax2, datax3, datax4))


xy_bootstrap(data_x, datay, 10000)

			XY BOOTSTRAP
╒════╤═════════════════════╤═════════════════════════╤═══════════╕
│    │  True Coefficients  │  Boostrap Coefficients  │   Bias    │
╞════╪═════════════════════╪═════════════════════════╪═══════════╡
│ β0 │       62.4054       │         82.0044         │  19.599   │
├────┼─────────────────────┼─────────────────────────┼───────────┤
│ β1 │       1.5511        │         1.37764         │ -0.173467 │
├────┼─────────────────────┼─────────────────────────┼───────────┤
│ β2 │      0.510168       │         0.30597         │ -0.204198 │
├────┼─────────────────────┼─────────────────────────┼───────────┤
│ β3 │      0.101909       │       -0.0739777        │ -0.175887 │
├────┼─────────────────────┼─────────────────────────┼───────────┤
│ β4 │      -0.144061      │        -0.350558        │ -0.206497 │
╘════╧═════════════════════╧═════════════════════════╧═══════════╛


## _BOOTSTRAP - Residual_

In [6]:
def resiudal_bootstrap(DataX, DataY, n_bootstrap):
    banyak_baris = DataY.shape[0]
    banyak_kolom = DataX.shape[1]
    matriks_1 = []
    for i in range(banyak_baris):
        matriks_1.append(1)
    matriks_x = np.column_stack((matriks_1, DataX))
    b = np.linalg.inv(matriks_x.T @ matriks_x) @ (matriks_x.T @ DataY.T)
    ypred = matriks_x@b
    e = DataY-ypred

    matriks_bootstrap = np.empty([n_bootstrap, len(b)])
    for i in range(n_bootstrap):
        indeks_acak = np.random.choice(banyak_baris, banyak_baris, replace=True)
        e_sample = e[indeks_acak]
        y_bootstrap = e_sample + ypred
        b_bootstrap = np.linalg.inv(matriks_x.T @ matriks_x) @ (matriks_x.T @ y_bootstrap.T)

        for j in range(len(b)):
            matriks_bootstrap[i,j] = b_bootstrap[j]

    koefisien_bootstrap = []  
    for i in range(len(b)):
        koefisien_bootstrap.append(np.mean(matriks_bootstrap[:, i]))

    #OUTPUT
    nama_baris = []
    for i in range(len(b)):
        nama_baris.append(f'β{i}')
    nama_kolom = ["True Coefficients", "Boostrap Coefficients", "Bias"]
    bias = koefisien_bootstrap-b
    isi_tabel = np.column_stack((b, koefisien_bootstrap, bias))
    output_df = pd.DataFrame(isi_tabel, index = nama_baris, columns = nama_kolom)
    tabel_output = tabulate(output_df, headers='keys', tablefmt='fancy_grid', numalign="center")
    print("==================================================================")
    print("\t\t\tRESIDUAL BOOTSTRAP")
    print("==================================================================")
    print(tabel_output)

In [7]:
data = pd.read_excel("Data_Tugas_Anreg_M13.xlsx")
datax1 = data['X1']
datax2 = data['X2']
datax3 = data['X3']
datax4 = data['X4']
datay = data['Y']
data_x = np.column_stack((datax1, datax2, datax3, datax4))


resiudal_bootstrap(data_x, datay, 1000)

			RESIDUAL BOOTSTRAP
╒════╤═════════════════════╤═════════════════════════╤════════════╕
│    │  True Coefficients  │  Boostrap Coefficients  │    Bias    │
╞════╪═════════════════════╪═════════════════════════╪════════════╡
│ β0 │       62.4054       │         62.1309         │ -0.274475  │
├────┼─────────────────────┼─────────────────────────┼────────────┤
│ β1 │       1.5511        │         1.55371         │ 0.00261084 │
├────┼─────────────────────┼─────────────────────────┼────────────┤
│ β2 │      0.510168       │        0.512357         │ 0.00218929 │
├────┼─────────────────────┼─────────────────────────┼────────────┤
│ β3 │      0.101909       │        0.107273         │ 0.00536405 │
├────┼─────────────────────┼─────────────────────────┼────────────┤
│ β4 │      -0.144061      │        -0.141537        │ 0.00252429 │
╘════╧═════════════════════╧═════════════════════════╧════════════╛


## _JACKNIFE REGRESSION_

In [8]:
def jackknife_regresi(x, y):
    def hitung_beta_regresi(data_x, data_y):
        matriks_x = np.column_stack((np.ones(data_x.shape[0]), data_x))
        b = np.linalg.inv(matriks_x.T @ matriks_x) @ (matriks_x.T @ data_y)
        return b
    
    banyak_baris = y.shape[0]
    banyak_kolom = x.shape[1]
    
    true_b = hitung_beta_regresi(x, y)
    
    coefs_jackknife = []
    
    for i in range(banyak_baris):
        x_jackknife = np.delete(x, i, axis=0)
        y_jackknife = np.delete(y, i, axis=0)
        
        model = hitung_beta_regresi(x_jackknife, y_jackknife)
        
        coefs_jackknife.append(model)
        
    coef_mean = np.mean(coefs_jackknife, axis=0)
    
    # OUTPUT
    nama_baris = [f'β{i}' for i in range(len(true_b))]
    nama_kolom = ["True Coefficients", "Jackknife Coefficients", "Bias"]
    bias = coef_mean - true_b
    isi_tabel = np.column_stack((true_b, coef_mean, bias))
    output_df = pd.DataFrame(isi_tabel, index=nama_baris, columns=nama_kolom)
    tabel_output = tabulate(output_df, headers='keys', tablefmt='fancy_grid', numalign="center")
    
    print("==================================================================")
    print("\t\t\tJACKKNIFE REGRESSION")
    print("==================================================================")
    print(tabel_output)

In [9]:
data = pd.read_excel("Data_Tugas_Anreg_M13.xlsx")
datax1 = data['X1']
datax2 = data['X2']
datax3 = data['X3']
datax4 = data['X4']
datay = data['Y']
data_x = np.column_stack((datax1, datax2, datax3, datax4))


jackknife_regresi(data_x, datay)

			JACKKNIFE REGRESSION
╒════╤═════════════════════╤══════════════════════════╤════════════╕
│    │  True Coefficients  │  Jackknife Coefficients  │    Bias    │
╞════╪═════════════════════╪══════════════════════════╪════════════╡
│ β0 │       62.4054       │         65.0113          │  2.60592   │
├────┼─────────────────────┼──────────────────────────┼────────────┤
│ β1 │       1.5511        │         1.52348          │ -0.0276193 │
├────┼─────────────────────┼──────────────────────────┼────────────┤
│ β2 │      0.510168       │         0.483921         │ -0.0262466 │
├────┼─────────────────────┼──────────────────────────┼────────────┤
│ β3 │      0.101909       │        0.0737863         │ -0.0281231 │
├────┼─────────────────────┼──────────────────────────┼────────────┤
│ β4 │      -0.144061      │        -0.170396         │ -0.0263346 │
╘════╧═════════════════════╧══════════════════════════╧════════════╛


In [4]:
data = pd.read_excel("Data_Tugas_Anreg_M13.xlsx")
datax1 = data['X1']
datax2 = data['X2']
datax3 = data['X3']
datax4 = data['X4']
datay = data['Y']
data_x = np.column_stack((datax1, datax2, datax3, datax4))
xy = np.column_stack((data_x,datay))
xy
xy = xy[np.random.choice(xy.shape[0], xy.shape[0], replace=True)]
xy

array([[  2. ,  54. ,  18. ,  22. ,  93.1],
       [  1. ,  31. ,  22. ,  44. ,  72.5],
       [  3. ,  71. ,  17. ,   6. , 102.7],
       [ 11. ,  55. ,   9. ,  22. , 109.2],
       [ 11. ,  31. ,   8. ,  47. ,  87.6],
       [  2. ,  54. ,  18. ,  22. ,  93.1],
       [ 21. ,  47. ,   4. ,  26. , 115.9],
       [ 11. ,  66. ,   9. ,  12. , 113.3],
       [  1. ,  31. ,  22. ,  44. ,  72.5],
       [ 11. ,  56. ,   8. ,  20. , 104.3],
       [  1. ,  29. ,  15. ,  52. ,  74.3],
       [  3. ,  71. ,  17. ,   6. , 102.7],
       [  2. ,  54. ,  18. ,  22. ,  93.1]])